表4-1 XPath常用规则

表达式

描述
nodename     选取此节点的所有子节点

/            从当前节点选取直接子节点

//           从当前节点选取子孙节点

.            选取当前节点

..           选取当前节点的父节点

@            选取属性

这里列出了XPath的常用匹配规则，示例如下：

//title[@lang='eng']


这就是一个XPath规则，它代表选择所有名称为title，同时属性lang的值为eng的节点。

In [1]:
from lxml import etree
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result.decode('utf-8'))

<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


调用tostring()方法即可输出修正后的HTML代码，
但是结果是bytes类型。这里利用decode()方法将其转成str类型

5. 所有节点
我们一般会用//开头的XPath规则来选取所有符合要求的节点。这里以前面的HTML文本为例，如果要选取所有节点，可以这样实现：

In [2]:
from lxml import etree
html = etree.HTML(text)
result  = html.xpath('//*')
print(result)
result = html.xpath('//li')
print(result)
print(result[0])

[<Element html at 0x56adfc8>, <Element body at 0x5c478c8>, <Element div at 0x56b0b88>, <Element ul at 0x56ad7c8>, <Element li at 0x56ad708>, <Element a at 0x56adcc8>, <Element li at 0x56ad9c8>, <Element a at 0x56ada08>, <Element li at 0x56add08>, <Element a at 0x56adb88>, <Element li at 0x56ad088>, <Element a at 0x56adc88>, <Element li at 0x56adc48>, <Element a at 0x56adc08>]
[<Element li at 0x56ad708>, <Element li at 0x56ad9c8>, <Element li at 0x56add08>, <Element li at 0x56ad088>, <Element li at 0x56adc48>]
<Element li at 0x56ad708>


6. 子节点

我们通过/或//即可查找元素的子节点或子孙节点。假如现在想选择li节点的所有直接a子节点，可以这样实现：

In [3]:
## /用于选取直接子节点，如果要获取所有子孙节点，就可以使用//
result1 = html.xpath('//li/a')
print(result1)
result2 = html.xpath('//li//a')
print(result2)

[<Element a at 0x5c44988>, <Element a at 0x56ac488>, <Element a at 0x56ac2c8>, <Element a at 0x56ac208>, <Element a at 0x56ac388>]
[<Element a at 0x5c44988>, <Element a at 0x56ac488>, <Element a at 0x56ac2c8>, <Element a at 0x56ac208>, <Element a at 0x56ac388>]


7. 父节点

我们知道通过连续的/或//可以查找子节点或子孙节点，那么假如我们知道了子节点，怎样来查找父节点呢？这可以用..来实现。

比如，现在首先选中href属性为link4.html的a节点，然后再获取其父节点，然后再获取其class属性，相关代码如下：

In [5]:
result = html.xpath('//a[@href="link4.html"]/../@class')
print(result)


result2 = html.xpath('//a[@href="link4.html"]/parent::*/@class')
print(result2)

['item-1']
['item-1']


8. 属性匹配

在选取的时候，我们还可以用@符号进行属性过滤。比如，这里如果要选取class为item-1的li节点，可以这样实现:

In [6]:
result = html.xpath('//li[@class="item-0"]')
print(result)

[<Element li at 0x56ad708>, <Element li at 0x56adc48>]


我们用XPath中的text()方法获取节点中的文本，接下来尝试获取前面li节点中的文本，相关代码如下：

In [10]:
result = html.xpath('//li[@class="item-0"]/text()')
print(result)

result2 = html.xpath('//li[@class="item-0"]//text()')
print(result2)

result2 = html.xpath('//li[@class="item-0"]/a/text()')
print(result2)

['\n     ']
['first item', 'fifth item', '\n     ']
['first item', 'fifth item']


属性获取

我们知道用text()可以获取节点内部文本，那么节点属性该怎样获取呢？其实还是用@符号就可以。例如，我们想获取所有li节点下所有a节点的href属性，代码如下：

In [11]:
result = html.xpath('//li/a/@href')
print(result)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']


属性多值匹配

有时候，某些节点的某个属性可能有多个值

In [17]:
text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[@class="li"]//text()')
print(result)

result = html.xpath('//li[contains(@class,"li")]//text()')
print(result)

[]
['first item']


多属性匹配

另外，我们可能还遇到一种情况，那就是根据多个属性确定一个节点，这时就需要同时匹配多个属性。此时可以使用运算符and来连接

In [19]:
text = '''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''

html = etree.HTML(text)
result = html.xpath('//li[contains(@class,"li")and @name="item"]/a/text()')
print(result)

['first item']


 按序选择
 
有时候，我们在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，这时该怎么办呢？

In [20]:
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
    
html = etree.HTML(text)
result = html.xpath('//li[1]/a/text()')
print(result)
result = html.xpath('//li[last()]/a/text()')
print(result)
result = html.xpath('//li[position()<3]/a/text()')
print(result)
result = html.xpath('//li[last()-2]/a/text()')
print(result)

['first item']
['fifth item']
['first item', 'second item']
['third item']


节点轴选择

XPath提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等，示例如下：

In [26]:
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''

html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*') #所有祖先节点
print(result)

result = html.xpath('//li[1]/ancestor::div') #有div这个祖先节点
print(result)
result = html.xpath('//li[1]/attribute::*')#我们调用了attribute轴，可以获取所有属性值
print(result)
result = html.xpath('//li[i]/child::a[@herf="link1.html"]') #我们调用了child轴，可以获取所有直接子节点
print(result)
result = html.xpath('//li[1]/descendant::span') #我们调用了descendant轴，可以获取所有子孙节点
print(result)
result = html.xpath('//li[1]/following::*[2]') #们调用了following轴，可以获取当前节点之后的所有节点。
print(result)
result = html.xpath('//li[1]/following-sibling::*') #我们调用了following-sibling轴，可以获取当前节点之后的所有同级节点
print(result)


[<Element html at 0x56b03c8>, <Element body at 0x7229cc8>, <Element div at 0x7229c48>, <Element ul at 0x7229bc8>]
[<Element div at 0x7229c48>]
['item-0']
[]
[<Element span at 0x7229ac8>]
[<Element a at 0x7229c48>]
[<Element li at 0x7229cc8>, <Element li at 0x7229a48>, <Element li at 0x74fd248>, <Element li at 0x74fd548>]
